In [26]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error, mean_absolute_error

pd.set_option('max_columns', None)

## Diary
21/09: Submission without handling label data got 0.16 MSE. Still need to make some progress

## Reading data

In [14]:
train_df = pd.read_csv('/home/tuts/datasets/house-prices-advanced-regression-techniques/train.csv')
test_df = pd.read_csv('/home/tuts/datasets/house-prices-advanced-regression-techniques/test.csv')

In [15]:
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [16]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [32]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

## Selecting columns
### Later, I need to handle missing values.

In [30]:
# train_df = train_df.drop(labels=['PoolQC', 'Fence', 'MiscFeature', 'Alley', 'Id', 'FireplaceQu'], axis=1)
use_cols = ['LotArea', 'YrSold', 'MoSold', 'MiscVal','PoolArea','ScreenPorch',
              'OpenPorchSF','WoodDeckSF','GarageArea','GarageCars','Fireplaces', 
              'KitchenAbvGr', 'BedroomAbvGr', 'FullBath', 'HalfBath', 'GrLivArea',
              'YearBuilt', 'OverallQual', 'OverallCond', 'LotArea', 'MSSubClass']
X = train_df[use_cols]
X.head()

,LotArea,YrSold,MoSold,MiscVal,PoolArea,ScreenPorch,OpenPorchSF,WoodDeckSF,GarageArea,GarageCars,Fireplaces,KitchenAbvGr,BedroomAbvGr,FullBath,HalfBath,GrLivArea,YearBuilt,OverallQual,OverallCond,LotArea,MSSubClass
0,8450,2008,2,0,0,0,61,0,548,2,0,1,3,2,1,1710,2003,7,5,8450,60
1,9600,2007,5,0,0,0,0,298,460,2,1,1,3,2,0,1262,1976,6,8,9600,20
2,11250,2008,9,0,0,0,42,0,608,2,1,1,3,2,1,1786,2001,7,5,11250,60
3,9550,2006,2,0,0,0,35,0,642,3,1,1,3,1,0,1717,1915,7,5,9550,70
4,14260,2008,12,0,0,0,84,192,836,3,1,1,4,2,1,2198,2000,8,5,14260,60


In [18]:
y = train_df.SalePrice

## Encoding (skipped for now)

In [60]:
label_columns = list(train_df.dtypes[train_df.dtypes == object].index)

In [61]:
train_df.isna().sum().nunique()

1

In [44]:
for column in label_columns:
    if train_df[column].nunique() > 6:
        le = LabelEncoder()
        df = pd.DataFrame()
        if train_df[column].dtype == object:
            df = pd.DataFrame(le.fit_transform(train_df[column].astype(str)), columns=[column])
        else:
            df = pd.DataFrame(le.fit_transform(train_df[column], columns=[column]))
        train_df = train_df.drop([column], axis=1)
        train_df = pd.concat([train_df, df], axis=1)
    else:
        oh = pd.get_dummies(train_df[column])
        train_df = train_df.drop([column], axis=1)
        train_df = pd.concat([train_df, oh], axis=1)

train_df.shape

5
MSZoning
object
2
Street
object
4
LotShape
object
4
LandContour
object
1
Utilities
object
5
LotConfig
object
3
LandSlope
object
25
Neighborhood
object
9
Condition1
object
6
Condition2
object
5
BldgType
object
8
HouseStyle
object
5
RoofStyle
object
7
RoofMatl
object
14
Exterior1st
object
16
Exterior2nd
object
4
MasVnrType
object
4
ExterQual
object
4
ExterCond
object
5
Foundation
object
4
BsmtQual
object
4
BsmtCond
object
4
BsmtExposure
object
6
BsmtFinType1
object
6
BsmtFinType2
object
4
Heating
object
5
HeatingQC
object
2
CentralAir
object
5
Electrical
object
4
KitchenQual
object
6
Functional
object
6
GarageType
object
3
GarageFinish
object
5
GarageQual
object
5
GarageCond
object
3
PavedDrive
object
9
SaleType
object
6
SaleCondition
object


(1460, 179)

In [45]:
train_df.columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       ...
       'N', 'P', 'Y', 'SaleType', 'Abnorml', 'AdjLand', 'Alloca', 'Family',
       'Normal', 'Partial'],
      dtype='object', length=179)

In [6]:
train_df.shape

(1094, 75)

## Training

In [20]:
l1 = Lasso(normalize=True)
l2 = Ridge(normalize=True, solver='lsqr')
rf = RandomForestRegressor()
ab = AdaBoostRegressor()

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [22]:
X_train.describe()

,LotArea,YrSold,MoSold,MiscVal,PoolArea,ScreenPorch,OpenPorchSF,WoodDeckSF,GarageArea,GarageCars,Fireplaces,KitchenAbvGr,BedroomAbvGr,FullBath,HalfBath,GrLivArea,YearBuilt,OverallQual,OverallCond,LotArea,MSSubClass
count,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000
mean,10708.294521,2007.848337,6.335616,38.634051,3.941292,16.333659,47.458904,94.864971,469.704501,1.758317,0.613503,1.040117,2.865949,1.549902,0.385519,1520.013699,1971.020548,6.114481,5.628180,10708.294521,56.570450
std,10949.370637,1.329796,2.669867,507.355485,47.979534,59.531629,66.781895,126.219207,214.214468,0.748033,0.644675,0.206067,0.806824,0.551869,0.500837,538.574101,29.790923,1.366883,1.123478,10949.370637,42.329388
min,1477.000000,2006.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,480.000000,1872.000000,2.000000,2.000000,1477.000000,20.000000
25%,7588.500000,2007.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,308.000000,1.000000,0.000000,1.000000,2.000000,1.000000,0.000000,1124.250000,1954.000000,5.000000,5.000000,7588.500000,20.000000
50%,9501.500000,2008.000000,6.000000,0.000000,0.000000,0.000000,26.000000,0.000000,474.000000,2.000000,1.000000,1.000000,3.000000,2.000000,0.000000,1470.000000,1972.000000,6.000000,5.000000,9501.500000,50.000000
75%,11644.500000,2009.000000,8.000000,0.000000,0.000000,0.000000,71.500000,168.000000,576.000000,2.000000,1.000000,1.000000,3.000000,2.000000,1.000000,1785.500000,2000.000000,7.000000,6.000000,11644.500000,70.000000
max,215245.000000,2010.000000,12.000000,15500.000000,738.000000,480.000000,547.000000,857.000000,1418.000000,4.000000,3.000000,3.000000,8.000000,3.000000,2.000000,5642.000000,2010.000000,10.000000,9.000000,215245.000000,190.000000


In [23]:
print(X_train.isnull().values.any())
print(y_train.isnull().values.any())

False
False


In [24]:
rf.fit(X_train, y_train)

RandomForestRegressor()

In [27]:
yhat = rf.predict(X_test)
print("MSE {}".format(np.sqrt(mean_squared_error(y_test, yhat)))
print("MAE {}".format(mean_absolute_error(y_test, yhat))

816672506.4433916

In [33]:
X_test = test_df[use_cols]
X_test.info()

In [35]:
X_test.isna().sum()

LotArea         0
YrSold          0
MoSold          0
MiscVal         0
PoolArea        0
ScreenPorch     0
OpenPorchSF     0
WoodDeckSF      0
GarageArea      1
GarageCars      1
Fireplaces      0
KitchenAbvGr    0
BedroomAbvGr    0
FullBath        0
HalfBath        0
GrLivArea       0
YearBuilt       0
OverallQual     0
OverallCond     0
LotArea         0
MSSubClass      0
dtype: int64

In [38]:
garage_area_med = X_test.GarageArea.median()
garage_cars_med = X_test.GarageCars.median()

X_test.GarageArea = X_test.GarageArea.fillna(garage_area_med)
X_test.GarageCars = X_test.GarageCars.fillna(garage_cars_med)

y_pred = rf.predict(X_test)

## Submission

In [46]:
my_submission = pd.DataFrame({'Id': test_df.Id, 'SalePrice': y_pred})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)